In [1]:
# Required Libraries
import numpy as np
from sympy import *
import pandas as pd

In [2]:

from bokeh.layouts import column, row, widgetbox
from bokeh.models import CustomJS, Slider, LinearAxis, Range1d,Label, LabelSet
from bokeh.plotting import ColumnDataSource, figure, output_file, show
from bokeh.io import show, output_notebook, curdoc
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn

from bokeh.layouts import row, column
from bokeh.models import CustomJS, Slider
from bokeh.plotting import figure, output_file, show, ColumnDataSource

# Import data

Expected Utility, spread

In [3]:
# 
def util_endpoints(H,alpha, mu, delta,p, gamma):
  '''
   This function computes utility endpoints for 
   both agent types (bandit and market maker)
   Inputs: initial parameters:
      alpha: news arrival
      mu: LT arrival
      delta: Exchange latency
      gamma : Risk averion factor
      p: sniping probability 
  Outputs:
      A,B : Utility endpoints for Bandits
      C,D : Utility endpoints for Market maker
      u_star : optimal utility
      s_star : optimal spread
   '''
  # initial parametres 
  beta = alpha/(alpha + mu) 
  alphabar  = alpha  * delta /2 # rate of news arrival
  mubar = mu * delta/2  # rate of LT arrival
  q = gamma -1 
  thetabar = 2*alphabar*mubar/(alphabar +mubar)
  m = 1- mubar
  h = ( p*H - 1 + (1-p)**H)/ (p*H)
  g = h/((H-1)*p)
  
  A = beta*g*p*(-mubar + 1)
  B = alphabar*beta*g*p*(-gamma + 1)
  C =  alphabar*beta*q - alphabar*q + 2*beta*h*mubar*q + beta*h*mubar - beta*h*q - beta*h - beta*mubar*q
  D =  -alphabar*beta*h*q + beta*mubar - beta + mubar + 1

  # Computing optimal utility (U*(p))
  N =  A - C + D - B
  Q = A*D - B*C

  u_star =( A *D - B*C)/N
  
  # Computing optimal spread (s*(p))
  s_star = (A - C)/ N

  return A, B, C, D, s_star, u_star

#------------- Extract util_endpoints (as a function of p (probability)) -------------
p = Symbol('p')
res = util_as_p = util_endpoints(5,.45,.5,.5,p,1.5)
A = util_as_p[0]
B = util_as_p[1]
C = util_as_p[2]
D = util_as_p[3]
#------------ Compute s**
# s** : the spread when bandit is indifferent in sniping (bandit utility pivot around s**)
# Bandit Utility to compute s** when p
s = Symbol('s')
EU_B = (1-s)*A+(s*B)
s_2star = solve(EU_B,s)
s_2star = round(s_2star[0],2)

#----------- convert symbolic expressions to py function
A = lambdify(p,A)
B = lambdify(p,B)
C = lambdify(p,C)
D = lambdify(p,D)


# Bokeh Plot:


1.   Create data source
2.   Define Plot template
3.   Update Dataset (callback function)
4.   Create a slider and widget 
5.   Apply updates on plot 

![alt text](https://drive.google.com/uc?export=view&id=1IawPtKVlb3F1cUNwJ8KEoUspnAqwJhOB)



In [4]:
# -------------------------------------------------
# Build the range of probability 

nr_steps = 100
START_P = 0.5

P_approx = []

# Together with the mapping used in JS callabk to convert a P into its index
P_values = {}
for i in range(nr_steps):
  P_approx.append((i+1)/nr_steps)
  P_values[(i+1)/nr_steps] = i

# --------------------
# Build P and the 4 vectors (2 sets of 2 points that defines a set of 2 lines)
P = np.array(P_approx)

# Line 1: Bandit (y1, y2)
A_p = A(P).tolist()
B_p = B(P).tolist()
# Line 2: Market maker (y1, y2)
C_p = C(P).tolist()
D_p = D(P).tolist()

# -------------------------------------------------

# Build the source of the 1st preview
start_index = P_values[START_P]
source = ColumnDataSource(data=dict(
    x=[0,1],
    y1=[A_p[start_index], B_p[start_index]],
    y2=[C_p[start_index], D_p[start_index]]
))
# -------------------------------------------------
# Define the plot
plot = figure(plot_width = 500, plot_height = 400, title = 'Sniping probability on changes in utility',x_axis_label ='spread', y_axis_label = 'utility', y_range=(-0.5,1), x_range = (0,1))


# Draw lines
plot.vline_stack(['y1', 'y2'], x='x', source=source, color=["red", "navy"])
plot.line((0,1), (0,0), line_color="black") # Add horizontal line for xaxis 

#source_1ables = ColumnDataSource(data=dict(xx=[0, 1, 0, 1],yy=[A_p[0], B_p[0], C_p[0], D_p[0]], points=['A(p)', 'B(p)', 'C(p)', 'D(p)']))
#labels = LabelSet(x='xx', y='yy', text='points', level='glyph',x_offset=4, y_offset=4, source=source_1ables, render_mode='canvas')

labels = Label(x = s_2star, y = 0, text='s**', render_mode='canvas') # Add label for s**

#---------------------------------------------------
# Update data and plot by JS Callback for slider
callback = CustomJS(args=dict(source=source, A_p=A_p, B_p=B_p, C_p=C_p, D_p=D_p, P_values=P_values), code="""
        // data is an array of 3 arrays data['x'], data['y1'] and data['y2']
        var data = source.getv('data');

        // f is the Slider position between 0 (excluded) and 1
        var f = Number(cb_obj.getv('value').toFixed(2));
        // Handle the weird number conversion (and / or format) for 1
        if (f == 1) {
          f = "1.0";
        }

        // Update values for the two points of y1 and y2
        data['y1'] = [A_p[P_values[f]], B_p[P_values[f]]]
        data['y2'] = [C_p[P_values[f]], D_p[P_values[f]]]

        // Callback
        source.change.emit();
    """)

#-----------------------------------------------------
# create Slider
slider_p = Slider(title = 'Sniping probability', start = P[0], end = P[len(P)-1], step = 1/nr_steps, value = START_P)
slider_p.js_on_change('value', callback)

# Create a widgetbox layout: layout
layout = column(slider_p, plot)

plot.add_layout(labels)

# Add the layout to the current document
curdoc().add_root(layout)
output_notebook()
show(layout)